## Load model and prepare test data for prediction

In [1]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers BASE_DIR = ''
import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.models import load_model

Using TensorFlow backend.


In [2]:
model = load_model('mymodel_large.h5')

In [3]:
test  = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [4]:
train['author'] = pd.Categorical(train['author'])
train['labels'] = train['author'].cat.codes
labels = train['labels']
labels_index = dict(enumerate(train['author'].cat.categories))
labels_index

{0: 'EAP', 1: 'HPL', 2: 'MWS'}

In [5]:
test_texts = test['text']

In [7]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = len(test_texts)
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [8]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(test_texts)
sequences = tokenizer.texts_to_sequences(test_texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)

Found 18042 unique tokens.
Shape of data tensor: (8392, 1000)


In [9]:
predictions = model.predict(data, batch_size=128, verbose=2)

In [10]:
submission = pd.DataFrame(predictions, index=test.id).rename(columns=labels_index)

In [11]:
submission.head()

,EAP,HPL,MWS
id,,,
id02310,0.616964,0.077819,0.305216
id24541,0.205602,0.012982,0.781415
id00134,0.504834,0.331115,0.164051
id27757,0.382602,0.160759,0.456638
id04081,0.100104,0.052366,0.847530


In [12]:
submission.to_csv('submissions/word_embeddings.csv')